In [206]:
import funct
import importlib
funct = importlib.reload(funct)
import numpy as np
import pandas as pd
import os
import netCDF4 as nc
import IPython

Read historical CCS data

In [207]:
freq = 3

# Get the current working directory
current_dir = "/".join(
        IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[-5:]
    )
# Get the parent directory path
parent_dir = os.path.dirname(os.path.dirname(current_dir))
# Change the directory (optional)
os.chdir(parent_dir)
# Now, your current working directory is the parent folder.

rela_path_base = "Data\Persian_CCS_201901_202403\CCS_esmJSON_2024-03-08075032am_"
df_list = []
for month in funct.iterate_months(2019, 1, 2024, 3):
    rela_path = rela_path_base + month + ".nc"
    file_path = os.path.join(parent_dir,rela_path)
    df = funct.create_time_series_dataframe(freq, rela_path)
    df_list.append(df)
df_ccs = pd.concat(df_list)
# orginal shape of the Esmeraldas basin is of size 53*52
# New flat array of 2756 pixels. Matches the original dimensions
df_ccs = df_ccs.replace(-99,np.nan)

Read and organize data from Esmeraldas station

In [171]:
# Get the current working directory
current_dir = "/".join(
        IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[-5:]
    )
# Get the parent directory path
parent_dir = os.path.dirname(os.path.dirname(current_dir))
# Change the directory (optional)
os.chdir(parent_dir)
# Now, your current working directory is the parent folder.

from datetime import date, timedelta
from funct import format_number_with_zeros
import chardet
import re


# Define start and end dates (replace with your desired dates)
start_date = date(2023, 9, 6)  # Adjust year, month, day as needed
end_date = date(2024, 3, 8)  # Adjust year, month, day as needed

# Iterate through each day using a for loop and timedelta

df_list_esm = []

for day in range((end_date - start_date).days + 1):
    current_date = start_date + timedelta(days=day)
    # Do something with the current_date here (e.g., print, process data)
    year = current_date.year
    month = format_number_with_zeros(current_date.month,2)
    day = format_number_with_zeros(current_date.day,2)
    csv_path = "Data\Esm_Station\yd" + str(year) + "\md" + str(year) + str(month) + "\\" + str(year) + str(month) + str(day) + "_dvd.csv"
    with open(csv_path, 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding']
    df = pd.read_csv(csv_path, encoding= encoding, header = None ,skiprows=2)
    df = df[0].apply(lambda x: pd.Series(re.split(r';+', x)))
    df = df.T
    df.columns = df.iloc[0]
    df = df.iloc[1:]
    df = df.set_index('N Sens', drop = True)
    df_list_esm.append(df)

df_list_esm

# Drafts Below